In [5]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd


import findspark
findspark.init()
import pyspark # Call this only after findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

print(spark)



sparkSession = SparkSession.builder.appName("test_read_from_pg").getOrCreate()


data="hdfs://namenode:9000/user/test/input/tweets_pg_export.txt"

df_load = sparkSession.read.csv(data)
df_load.show(2, False)

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#conver Spark dataframe into Pandas dataframe
df_pandas = df_load.select("*").toPandas()

print(df_pandas)

+------------------------------------------------+
|_c0                                             |
+------------------------------------------------+
|"@flawarah ""How Haze Affects Milk Tea Brand."""|
|The haze are really bad                         |
+------------------------------------------------+
only showing top 2 rows

                                                    _c0
0      "@flawarah ""How Haze Affects Milk Tea Brand."""
1                               The haze are really bad
2     RT @allyxjackson: do yall understand just how ...
3     RT @BRITgrlINDOfood: Standing on a bridge over...
4        The haze is worse than when I arrived yester…"
...                                                 ...
4941                          @knowyouskz Sometime okay
4942                   @techinsider POLLUTION REDUCTION
4943  this is still the funniest shit ever esgdjgash...
4944  The weather and the haze also makes me feel th...
4945                  @techinsider  POLLUTION REDUCTI